In [1]:
%load_ext Cython

In [2]:
def pairwise_manhattan_python(points):
    # Preallocate the results
    results = [[None] * len(points) for _ in range(len(points))]
    # Look at every pair of you and the ones after you, one before you were calculated when they looked at you
    for i in range(len(points)):
        for j in range(i, len(points)):
            # Manhattan distance calculation
            dist = sum(abs(p1 - p2) for p1, p2 in zip(points[i], points[j]))
            # Manhattan distance is symmetric so only calculate one for each pair
            results[i][j] = dist
            results[j][i] = dist
    return results

In [3]:
%%cython -a

def pairwise_manhattan_cython_untyped(points):
    results = [[None] * len(points) for _ in range(len(points))]
    for i in range(len(points)):
        for j in range(i, len(points)):
            dist = sum(abs(p1 - p2) for p1, p2 in zip(points[i], points[j]))
            results[i][j] = dist
            results[j][i] = dist
    return results

In [4]:
%%cython -a
from libc.stdlib cimport abs
cimport cython
from cython.parallel cimport prange
import numpy as np

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef pairwise_manhattan(int[:, :] points):
    cdef int n = points.shape[0]
    cdef int m = points.shape[1]
    results = np.zeros((n, n), dtype=np.int32)
    cdef int[:, :] results_view = results
    cdef int i, j, k, dist
    for i in range(n):
        for j in range(i, n):
            dist = 0
            for k in range(m):
                dist += abs(points[i, k] - points[j, k])
            results_view[i, j] = dist
            results_view[j, i] = dist
    return results

In [5]:
points = np.random.randint(-100, 100, size=[200, 100], dtype=np.int32)

In [6]:
%%timeit
pairwise_manhattan_python(points)

1.28 s ± 25.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
pairwise_manhattan_cython_untyped(points)

1.08 s ± 15.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
pairwise_manhattan(points)

1.05 ms ± 10.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
